# Testing hexagonal kernels and backward gradients

In [1]:
import numpy as np
import tensorflow as tf

import os
import tensorflow as tf

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0'
tf.logging.set_verbosity(tf.logging.DEBUG)

## Hexagonal layers

Test out the basic hexagonal layers.

In [2]:
from tensorflow.contrib import hex_layers

In [3]:
size = 6
shape = [size, size]
a = np.reshape(np.linspace(1.0, size**2, num=size**2, dtype=np.float32), shape)

print('Test Tensor: \n', a)
a = a.reshape([1, size, size, 1])

Test Tensor: 
 [[  1.   2.   3.   4.   5.   6.]
 [  7.   8.   9.  10.  11.  12.]
 [ 13.  14.  15.  16.  17.  18.]
 [ 19.  20.  21.  22.  23.  24.]
 [ 25.  26.  27.  28.  29.  30.]
 [ 31.  32.  33.  34.  35.  36.]]


In [4]:
t = hex_layers.hex_depthwise_conv2d(a, 3)
t = hex_layers.hex_downscale2d(a, 2)

In [5]:
# Test cartesian to hexagonal tiling.
t = hex_layers.hex_from_cartesian(a, downscale=False)
with tf.Session(config=tf.ConfigProto(log_device_placement=False)) as sess:
    print(t.eval().squeeze())



[[  1.    2.    3.    4.    5.    6. ]
 [  7.5   8.5   9.5  10.5  11.5  12. ]
 [ 13.   14.   15.   16.   17.   18. ]
 [ 19.5  20.5  21.5  22.5  23.5  24. ]
 [ 25.   26.   27.   28.   29.   30. ]
 [ 31.5  32.5  33.5  34.5  35.5  36. ]]


In [6]:
a.shape
t.get_shape()


TensorShape([Dimension(1), Dimension(6), Dimension(6), Dimension(1)])

In [7]:
aa = tf.convert_to_tensor(a)
l0 = aa[:,0]
print(l0.get_shape())

# aa.

(1, 6, 1)


In [26]:
5 % 3

2

## Forward hexagonal depthwise convolution 2D

Testing the forward convolution.

In [23]:
size = 5
shape = [size, size]
a = np.reshape(np.linspace(1.0, size**2, num=size**2, dtype=np.float32), shape)

print('Test Tensor: \n', a)
a = a.reshape([1, size, size, 1])

Test Tensor: 
 [[  1.   2.   3.   4.   5.]
 [  6.   7.   8.   9.  10.]
 [ 11.  12.  13.  14.  15.]
 [ 16.  17.  18.  19.  20.]
 [ 21.  22.  23.  24.  25.]]


In [31]:
# Testing several simple configurations of kernels.
radius = 2
ksize = 2*radius+1
stride = 1

with tf.Session(config=tf.ConfigProto(log_device_placement=False)) as sess:
    idx = 0
    for rad in range(radius+1):
        rsize = max(1, rad*6)
        for i in range(rsize):
            # Basic kernel
            f = np.zeros(shape=[ksize**2], dtype=np.float32)
            f[idx] = 1.
            print('TEST kernel %i: \n' % idx, f)
            # Inverse kernel.
            delta = rad*3 if i < rad*3 else -rad*3
            fi = np.zeros(shape=[ksize**2], dtype=np.float32)
            fi[idx+delta] = 1.
            # Hex. depthwise conv2d
            f = np.reshape(f, [ksize, ksize, 1, 1])
            t = tf.nn.hex_depthwise_conv2d(a, f, [1, stride, stride, 1], 'SAME', data_format='NHWC')
            r = t.eval()
            # Inverse computation
            fi = np.reshape(fi, [ksize, ksize, 1, 1])
            ti = tf.nn.hex_depthwise_conv2d(r, fi, [1, stride, stride, 1], 'SAME', data_format='NHWC')
            ri = ti.eval()
            
            print('TEST result: \n', r.squeeze(), '\nInverse:\n', ri.squeeze(), '\n')
            idx += 1

#     for k in range(ksize**2):
#         # Basic kernel
#         f = np.zeros(shape=[ksize**2], dtype=np.float32)
#         f[k] = 1.
#         print('TEST kernel %i: \n' % k, f)
#         # Hex. depthwise conv2d
#         f = np.reshape(f, [ksize, ksize, 1, 1])
#         t = tf.nn.hex_depthwise_conv2d(a, f, [1, stride, stride, 1], 'SAME', data_format='NHWC')
#         r = t.eval()
#         print('TEST result: \n', r.squeeze(), '\n')


TEST kernel 0: 
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.]
TEST result: 
 [[  1.   2.   3.   4.   5.]
 [  6.   7.   8.   9.  10.]
 [ 11.  12.  13.  14.  15.]
 [ 16.  17.  18.  19.  20.]
 [ 21.  22.  23.  24.  25.]] 
Inverse:
 [[  1.   2.   3.   4.   5.]
 [  6.   7.   8.   9.  10.]
 [ 11.  12.  13.  14.  15.]
 [ 16.  17.  18.  19.  20.]
 [ 21.  22.  23.  24.  25.]] 

TEST kernel 1: 
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.]
TEST result: 
 [[  0.   0.   0.   0.   0.]
 [  1.   2.   3.   4.   5.]
 [  0.   6.   7.   8.   9.]
 [ 11.  12.  13.  14.  15.]
 [  0.  16.  17.  18.  19.]] 
Inverse:
 [[  1.   2.   3.   4.   5.]
 [  6.   7.   8.   9.   0.]
 [ 11.  12.  13.  14.  15.]
 [ 16.  17.  18.  19.   0.]
 [  0.   0.   0.   0.   0.]] 

TEST kernel 2: 
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.]
TEST result: 
 [[  0

## Backward hexagonal depthwise convolution 2D - Input

Testing the backward convolution, input gradient.

In [11]:
size = 3
shape = [size, size]
a = np.reshape(np.linspace(1.0, size**2, num=size**2, dtype=np.float32), shape)

print('Test Tensor: \n', a)
a = a.reshape([1, size, size, 1])

Test Tensor: 
 [[ 1.  2.  3.]
 [ 4.  5.  6.]
 [ 7.  8.  9.]]


In [12]:
# Testing several simple configurations of kernels.
insize = 5
ksize = 3
stride = 2

with tf.Session(config=tf.ConfigProto(log_device_placement=False)) as sess:
    for k in range(ksize**2):
        # Basic kernel
        f = np.zeros(shape=[ksize**2], dtype=np.float32)
        f[k] = 1.
        print('TEST kernel: \n', f)
        # Hex. depthwise conv2d
        f = np.reshape(f, [ksize, ksize, 1, 1])
        t = tf.nn.hex_depthwise_conv2d_native_backprop_input(
            [1, insize, insize, 1], f, a, [1, stride, stride, 1], 'SAME', data_format='NHWC')        
        print('TEST result: \n', t.eval().squeeze(), '\n')



TEST kernel: 
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.]
TEST result: 
 [[ 1.  0.  2.  0.  3.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  4.  0.  5.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 7.  0.  8.  0.  9.]] 

TEST kernel: 
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.]
TEST result: 
 [[ 1.  0.  2.  0.  3.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  4.  0.  5.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 7.  0.  8.  0.  9.]] 

TEST kernel: 
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.]
TEST result: 
 [[ 1.  0.  2.  0.  3.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  4.  0.  5.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 7.  0.  8.  0.  9.]] 

TEST kernel: 
 [ 0.  0.  0.  1.  0.  0.  0.  0.  0.]
TEST result: 
 [[ 1.  0.  2.  0.  3.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  4.  0.  5.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 7.  0.  8.  0.  9.]] 

TEST kernel: 
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.]
TEST result: 
 [[ 1.  0.  2.  0.  3.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  4.  0.  5.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 7.  0.  8.  0.  9.]] 

TEST kernel: 
 [ 0.  0.  0.  0.  0.  1.  0.  0.  0.]
TEST result: 
 [[

## Backward hexagonal depthwise convolution 2D - kernel

Testing the backward convolution, input gradient.

In [13]:
size = 3
shape = [size, size]
a = np.reshape(np.linspace(1.0, size**2, num=size**2, dtype=np.float32), shape)

print('Test Tensor: \n', a)
a = a.reshape([1, size, size, 1])

Test Tensor: 
 [[ 1.  2.  3.]
 [ 4.  5.  6.]
 [ 7.  8.  9.]]


In [14]:
# Testing several simple configurations of kernels.
ksize = 3

with tf.Session(config=tf.ConfigProto(log_device_placement=False)) as sess:
    for k in range(ksize**2):
        # Basic kernel
        f = np.zeros(shape=[ksize**2], dtype=np.float32)
        f[k] = 1.
        print('TEST out gradient: \n', f.reshape([3, 3]))
        # Hex. depthwise conv2d
        f = np.reshape(f, [1, ksize, ksize, 1])
        t = tf.nn.hex_depthwise_conv2d_native_backprop_filter(
            f, [3, 3, 1, 1], a, [1, 1, 1, 1], 'SAME', data_format='NHWC')        
        print('TEST result: \n', t.eval().squeeze().flatten(), '\n')


TEST out gradient: 
 [[ 1.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
TEST result: 
 [ 1.  4.  0.  0.  0.  0.  2.  0.  0.] 

TEST out gradient: 
 [[ 0.  1.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
TEST result: 
 [ 2.  5.  4.  1.  0.  0.  3.  0.  0.] 

TEST out gradient: 
 [[ 0.  0.  1.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
TEST result: 
 [ 3.  6.  5.  2.  0.  0.  0.  0.  0.] 

TEST out gradient: 
 [[ 0.  0.  0.]
 [ 1.  0.  0.]
 [ 0.  0.  0.]]
TEST result: 
 [ 4.  8.  7.  0.  1.  2.  5.  0.  0.] 

TEST out gradient: 
 [[ 0.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  0.]]
TEST result: 
 [ 5.  9.  8.  4.  2.  3.  6.  0.  0.] 

TEST out gradient: 
 [[ 0.  0.  0.]
 [ 0.  0.  1.]
 [ 0.  0.  0.]]
TEST result: 
 [ 6.  0.  9.  5.  3.  0.  0.  0.  0.] 

TEST out gradient: 
 [[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 1.  0.  0.]]
TEST result: 
 [ 7.  0.  0.  0.  0.  4.  8.  0.  0.] 

TEST out gradient: 
 [[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  1.  0.]]
TEST result: 
 [ 8.  0.  0.  7.  4.  5.  9.  0.  0.] 

TEST out gradient: 
 [[ 

## Help...

In [15]:
tf.nn.hex_depthwise_conv2d?

In [16]:
tf.nn.depthwise_conv2d_native_backprop_filter?

In [17]:
tf.nn.hex_depthwise_conv2d_native_backprop_input?

In [18]:
tf.nn.hex_depthwise_conv2d_native_backprop_filter?